
# Trabajo Práctico 2

Dentro de la carpeta de `data/` vamos a poder encontrar 3 datasets:

- Pokemon (Problema de clasificacion Binaria): La intención es tratar de predecir si basada en la información que tenemos podemos estimar si la carta que estamos viendo es Legendaria o no (Booleano).
- Heart Disease (Problema de clasificacion Binaria): A partir de este dataset vamos a tratar de determinar si una persona basado en ciertos datos tiene asociado una enfermedad cardiaca.
- Challenger USA Space Shuttle O-Ring Data Set (Regresion multivariada): En este dataset encontramos información sobre varios despegues del space shuttle, la intención de este problema es tratar de predecir cuanto será la cantidad de O-Rings que fallan dadas las condiciones de despegue.

> Para la presentación de sus trabajos prácticos **Elijan 1 de los 3 datasets**

# Importar librerías

Importar aquellas librerías que serán utilizadas en el trabajo.

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Cargar datos

Cargar los datos de entrenamiento.

Recordemos que para todos los datasets tenemos el archivo `columns.csv` que contiene el nombre y descripción de cada columna en el mismo orden que van a encontrar los datos.

En los casos de datos con extensión  `*.data` vamos a tener que leerlo utilizando `pd.read_data('myfile.data', sep = ',', header = 0, names = my_cols)`

De esta manera si quisieramos cargar datos por ejemplo de Nasa deberiamos poner algo asi


```python
## Importing python earlier

my_cols = pd.read_csv('./data/nasa_orings/columns.csv').column.values.tolist()

my_data = pd.read_csv('./data/nasa_orings/o-ring-erosion-or-blowby.data',
                       delimiter=',',
                       header=0,
                       names = my_cols
                    )
```

In [2]:
my_cols = pd.read_csv('./data/pokemon/columns.csv').column.values.tolist()

df = pd.read_csv('./data/pokemon/pokemon.csv',
                       delimiter=',',
                       header=0,                       
                    )

# Análisis exploratorio básico y preprocesamiento de los datos

Análisis de los datos para conocer los mismos, ver datos faltantes, decidir cómo tratarlos, ver distribuciones, relaciones, etc. Procesar los datos centrándolos, reescalando, codificando, reduciendo dimensiones, etc. según considere necesario.

Vamos a considerar:

- Para escalar datos: `sklearn.preprocessing.StandardScaler()`
- Para completar valores nulos: `sklearn.impute.SimpleImputer()` [Quickguide](https://scikit-learn.org/stable/modules/impute.html)
- Para reduccion de dimensiones (sobre todo para visualizar los datos): PCA o [T-SNE](https://builtin.com/data-science/tsne-python) (Solo para visualización)
- Para codificar variables categóricas: `sklearn.preprocessing.OneHotEncoder()`

In [3]:
print("Estructura features del dataset: {}".format(df.shape))

Estructura features del dataset: (801, 41)


In [4]:
df.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

In [5]:
df.describe()

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,generation,is_legendary
count,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,...,781.000000,801.000000,703.000000,801.000000,801.000000,801.000000,801.000000,781.000000,801.000000,801.000000
mean,0.996255,1.057116,0.968789,1.073970,1.068976,1.065543,1.135456,1.192884,0.985019,1.034020,...,1.163892,68.958801,55.155761,401.000000,71.305868,70.911361,66.334582,61.378105,3.690387,0.087391
std,0.597248,0.438142,0.353058,0.654962,0.522167,0.717251,0.691853,0.604488,0.558256,0.788896,...,1.080326,26.576015,20.261623,231.373075,32.353826,27.942501,28.907662,109.354766,1.930420,0.282583
min,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.250000,0.250000,0.000000,0.250000,...,0.100000,1.000000,0.000000,1.000000,10.000000,20.000000,5.000000,0.100000,1.000000,0.000000
25%,0.500000,1.000000,1.000000,0.500000,1.000000,0.500000,0.500000,1.000000,1.000000,0.500000,...,0.600000,50.000000,50.000000,201.000000,45.000000,50.000000,45.000000,9.000000,2.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,65.000000,50.000000,401.000000,65.000000,66.000000,65.000000,27.300000,4.000000,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,1.500000,80.000000,50.000000,601.000000,91.000000,90.000000,85.000000,64.800000,5.000000,0.000000
max,4.000000,4.000000,2.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,14.500000,255.000000,100.000000,801.000000,194.000000,230.000000,180.000000,999.900000,7.000000,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   abilities          801 non-null    object 
 1   against_bug        801 non-null    float64
 2   against_dark       801 non-null    float64
 3   against_dragon     801 non-null    float64
 4   against_electric   801 non-null    float64
 5   against_fairy      801 non-null    float64
 6   against_fight      801 non-null    float64
 7   against_fire       801 non-null    float64
 8   against_flying     801 non-null    float64
 9   against_ghost      801 non-null    float64
 10  against_grass      801 non-null    float64
 11  against_ground     801 non-null    float64
 12  against_ice        801 non-null    float64
 13  against_normal     801 non-null    float64
 14  against_poison     801 non-null    float64
 15  against_psychic    801 non-null    float64
 16  against_rock       801 non

## Preprocessing

In [7]:
#Identificar columnas con valores NaN
colsNa = pd.DataFrame(data=df.isna().sum()>0, columns=['Valor'])
mask = colsNa.loc[colsNa['Valor']].index
df[mask]

,height_m,percentage_male,type2,weight_kg
0,0.7,88.1,poison,6.9
1,1.0,88.1,poison,13.0
2,2.0,88.1,poison,100.0
3,0.6,88.1,NaN,8.5
4,1.1,88.1,NaN,19.0
...,...,...,...,...
796,9.2,NaN,flying,999.9
797,0.3,NaN,steel,0.1
798,5.5,NaN,dragon,888.0
799,2.4,NaN,NaN,230.0


In [8]:
df.dtypes

abilities             object
against_bug          float64
against_dark         float64
against_dragon       float64
against_electric     float64
against_fairy        float64
against_fight        float64
against_fire         float64
against_flying       float64
against_ghost        float64
against_grass        float64
against_ground       float64
against_ice          float64
against_normal       float64
against_poison       float64
against_psychic      float64
against_rock         float64
against_steel        float64
against_water        float64
attack                 int64
base_egg_steps         int64
base_happiness         int64
base_total             int64
capture_rate          object
classfication         object
defense                int64
experience_growth      int64
height_m             float64
hp                     int64
japanese_name         object
name                  object
percentage_male      float64
pokedex_number         int64
sp_attack              int64
sp_defense    

In [9]:

# dejamos al Label cómo tipo distinto, pero numérico, para separar después por tipos
df = df.astype({'is_legendary': 'int32'})

In [10]:
df.dtypes

abilities             object
against_bug          float64
against_dark         float64
against_dragon       float64
against_electric     float64
against_fairy        float64
against_fight        float64
against_fire         float64
against_flying       float64
against_ghost        float64
against_grass        float64
against_ground       float64
against_ice          float64
against_normal       float64
against_poison       float64
against_psychic      float64
against_rock         float64
against_steel        float64
against_water        float64
attack                 int64
base_egg_steps         int64
base_happiness         int64
base_total             int64
capture_rate          object
classfication         object
defense                int64
experience_growth      int64
height_m             float64
hp                     int64
japanese_name         object
name                  object
percentage_male      float64
pokedex_number         int64
sp_attack              int64
sp_defense    

In [12]:
#colsNumericas = ['height_m', 'percentage_male', 'weight_kg']
#colsObject = ['type2']
colsNumericas = df.select_dtypes(['float64', 'int64']).columns
colsObject = df.select_dtypes(['object']).columns

### Estandarización

In [13]:
# Estandarizamos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[colsNumericas])

df2 = df.copy(deep=True)
df2[colsNumericas] = X_scaled

df2.describe()

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,generation,is_legendary
count,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,...,7.810000e+02,8.010000e+02,7.030000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,7.810000e+02,8.010000e+02,801.000000
mean,-1.774139e-16,-2.256483e-16,-5.100650e-17,9.445905e-17,1.181743e-15,-3.309185e-18,-2.361823e-16,2.566958e-16,-3.619486e-16,1.750576e-16,...,-9.183151e-17,-6.521347e-17,1.176552e-16,-6.653022e-18,-9.037021e-17,2.045804e-16,-1.976848e-17,6.151716e-17,6.381357e-16,0.087391
std,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,...,1.000641e+00,1.000625e+00,1.000712e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000641e+00,1.000625e+00,0.282583
min,-1.250270e+00,-1.843286e+00,-2.745712e+00,-1.640770e+00,-1.569399e+00,-1.486522e+00,-1.280632e+00,-1.560781e+00,-1.765558e+00,-9.944402e-01,...,-9.854192e-01,-2.558746e+00,-2.724117e+00,-1.729890e+00,-1.896041e+00,-1.823143e+00,-2.123067e+00,-5.607197e-01,-1.394551e+00,0.000000
25%,-8.314219e-01,-1.304414e-01,8.845721e-02,-8.768890e-01,-1.321788e-01,-7.889798e-01,-9.190580e-01,-3.192858e-01,2.685259e-02,-6.773436e-01,...,-5.222994e-01,-7.138259e-01,-2.546406e-01,-8.649449e-01,-8.135762e-01,-7.488387e-01,-7.384863e-01,-4.792811e-01,-8.762049e-01,0.000000
50%,6.274882e-03,-1.304414e-01,8.845721e-02,-1.130085e-01,-1.321788e-01,-9.143806e-02,-1.959092e-01,-3.192858e-01,2.685259e-02,-4.315047e-02,...,-1.518037e-01,-1.490545e-01,-2.546406e-01,0.000000e+00,-1.950251e-01,-1.758765e-01,-4.619591e-02,-3.118286e-01,1.604866e-01,0.000000
75%,6.274882e-03,-1.304414e-01,8.845721e-02,-1.130085e-01,-1.321788e-01,-9.143806e-02,1.250388e+00,-3.192858e-01,2.685259e-02,-4.315047e-02,...,3.113161e-01,4.157169e-01,-2.546406e-01,8.649449e-01,6.090914e-01,6.835668e-01,6.460945e-01,3.131174e-02,6.788323e-01,0.000000
max,5.032456e+00,6.720938e+00,2.922626e+00,4.470274e+00,5.616701e+00,4.093813e+00,4.142984e+00,4.646694e+00,5.404084e+00,3.762008e+00,...,1.235243e+01,7.004716e+00,2.214836e+00,1.729890e+00,3.794630e+00,5.696986e+00,3.934474e+00,8.587859e+00,1.715524e+00,1.000000


### Missing

In [16]:
# Completar valores NaN

valueNum=df2[colsNumericas].values
valueObj=df2[colsObject].values

impNum = SimpleImputer(missing_values=np.nan, strategy='mean')
impObject = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

transformed_valuesNum = impNum.fit_transform(valueNum)
transformed_valuesObj = impObject.fit_transform(valueObj)

df3 = df2.copy(deep=True)

df3[colsNumericas] = transformed_valuesNum
df3[colsObject] = transformed_valuesObj

print(df.isna().sum().sum())
print(df2.isna().sum().sum())
print(df3.isna().sum().sum())


522
522
0


In [17]:
df3.describe()

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,generation,is_legendary
count,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,...,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,8.010000e+02,801.000000
mean,-1.774139e-16,-2.256483e-16,-5.100650e-17,9.445905e-17,1.181743e-15,-3.309185e-18,-2.361823e-16,2.566958e-16,-3.619486e-16,1.750576e-16,...,-8.953859e-17,-6.521347e-17,1.013298e-16,-6.653022e-18,-9.037021e-17,2.045804e-16,-1.976848e-17,5.998115e-17,6.381357e-16,0.087391
std,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,...,9.880536e-01,1.000625e+00,9.374167e-01,1.000625e+00,1.000625e+00,1.000625e+00,1.000625e+00,9.880536e-01,1.000625e+00,0.282583
min,-1.250270e+00,-1.843286e+00,-2.745712e+00,-1.640770e+00,-1.569399e+00,-1.486522e+00,-1.280632e+00,-1.560781e+00,-1.765558e+00,-9.944402e-01,...,-9.854192e-01,-2.558746e+00,-2.724117e+00,-1.729890e+00,-1.896041e+00,-1.823143e+00,-2.123067e+00,-5.607197e-01,-1.394551e+00,0.000000
25%,-8.314219e-01,-1.304414e-01,8.845721e-02,-8.768890e-01,-1.321788e-01,-7.889798e-01,-9.190580e-01,-3.192858e-01,2.685259e-02,-6.773436e-01,...,-5.222994e-01,-7.138259e-01,-2.546406e-01,-8.649449e-01,-8.135762e-01,-7.488387e-01,-7.384863e-01,-4.765360e-01,-8.762049e-01,0.000000
50%,6.274882e-03,-1.304414e-01,8.845721e-02,-1.130085e-01,-1.321788e-01,-9.143806e-02,-1.959092e-01,-3.192858e-01,2.685259e-02,-4.315047e-02,...,-1.518037e-01,-1.490545e-01,-2.546406e-01,0.000000e+00,-1.950251e-01,-1.758765e-01,-4.619591e-02,-3.008481e-01,1.604866e-01,0.000000
75%,6.274882e-03,-1.304414e-01,8.845721e-02,-1.130085e-01,-1.321788e-01,-9.143806e-02,1.250388e+00,-3.192858e-01,2.685259e-02,-4.315047e-02,...,3.113161e-01,4.157169e-01,-9.096564e-17,8.649449e-01,6.090914e-01,6.835668e-01,6.460945e-01,1.115389e-03,6.788323e-01,0.000000
max,5.032456e+00,6.720938e+00,2.922626e+00,4.470274e+00,5.616701e+00,4.093813e+00,4.142984e+00,4.646694e+00,5.404084e+00,3.762008e+00,...,1.235243e+01,7.004716e+00,2.214836e+00,1.729890e+00,3.794630e+00,5.696986e+00,3.934474e+00,8.587859e+00,1.715524e+00,1.000000


In [18]:
df3.describe(include='O')

,abilities,capture_rate,classfication,japanese_name,name,type1,type2
count,801,801,801,801,801,801,801
unique,482,34,588,801,801,18,18
top,['Levitate'],45,Dragon Pokémon,Fushigidaneフシギダネ,Bulbasaur,water,flying
freq,29,250,8,1,1,114,479


In [ ]:
La columna capture_rate debería ser solo numérico. Se busca la o las filas con 

In [52]:
pd.set_option('display.max_columns', None)
df.loc[pd.to_numeric(df['capture_rate'], errors='coerce').isna()]

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,against_ground,against_ice,against_normal,against_poison,against_psychic,against_rock,against_steel,against_water,attack,base_egg_steps,base_happiness,base_total,capture_rate,classfication,defense,experience_growth,height_m,hp,japanese_name,name,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
773,['Shields Down'],0.5,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0,2.0,0.5,0.5,1.0,2.0,2.0,2.0,100,6400,70,500,30 (Meteorite)255 (Core),Meteor Pokémon,60,1059860,0.3,60,Metenoメテノ,Minior,NaN,774,100,60,120,rock,flying,40.0,7,0


In [38]:
df3['capture_rate'].astype('float64')

ValueError: could not convert string to float: '30 (Meteorite)255 (Core)'

In [21]:
df3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   abilities          801 non-null    object 
 1   against_bug        801 non-null    float64
 2   against_dark       801 non-null    float64
 3   against_dragon     801 non-null    float64
 4   against_electric   801 non-null    float64
 5   against_fairy      801 non-null    float64
 6   against_fight      801 non-null    float64
 7   against_fire       801 non-null    float64
 8   against_flying     801 non-null    float64
 9   against_ghost      801 non-null    float64
 10  against_grass      801 non-null    float64
 11  against_ground     801 non-null    float64
 12  against_ice        801 non-null    float64
 13  against_normal     801 non-null    float64
 14  against_poison     801 non-null    float64
 15  against_psychic    801 non-null    float64
 16  against_rock       801 non

In [22]:
df3.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",0.006275,-0.130441,0.088457,-0.876889,-1.090325,-0.788980,1.250388,1.336041,0.026853,...,1.6271,1,65,65,45,grass,poison,-0.498497,1,False
1,"['Overgrow', 'Chlorophyll']",0.006275,-0.130441,0.088457,-0.876889,-1.090325,-0.788980,1.250388,1.336041,0.026853,...,1.6271,2,80,80,60,grass,poison,-0.442679,1,False
2,"['Overgrow', 'Chlorophyll']",0.006275,-0.130441,0.088457,-0.876889,-1.090325,-0.788980,1.250388,1.336041,0.026853,...,1.6271,3,122,120,80,grass,poison,0.353406,1,False
3,"['Blaze', 'Solar Power']",-0.831422,-0.130441,0.088457,-0.113009,-1.090325,-0.091438,-0.919058,-0.319286,0.026853,...,1.6271,4,60,50,65,fire,flying,-0.483856,1,False
4,"['Blaze', 'Solar Power']",-0.831422,-0.130441,0.088457,-0.113009,-1.090325,-0.091438,-0.919058,-0.319286,0.026853,...,1.6271,5,80,65,80,fire,flying,-0.387777,1,False


In [39]:
#split

from sklearn.model_selection import train_test_split



label = 'is_legendary'
df3[label] = df3[label] * 1
features = df3.loc[:, df3.columns != label].columns
features = df3[colsNumericas].columns

X = df3[features]
y = df3[label]
y=y.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_val, X_val, y_val, y_val = train_test_split(X_test, y_test, test_size=0.33, random_state=57)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(536, 21)
(265, 21)
(88, 21)
(536,)
(265,)
(88,)


# Modelos y evaluación
Probar diferentes modelos para predecir la variable objetivo. Calcular las métricas que considere relevantes. Comentar los resultados. 

In [40]:
from sklearn.linear_model import ElasticNet, ElasticNetCV, LogisticRegression, LogisticRegressionCV, SGDClassifier, SGDRegressor

In [41]:
y_train.value_counts()

0    492
1     44
Name: is_legendary, dtype: int64

In [44]:
# Seleccionar hiperparámetros
lr = 0.01
b = 16
epochs = 100
bias = True

# Hacer el fit del modelo con los HPs seleccionados
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)


LogisticRegression()

In [45]:
predictions = logistic_regression.predict(X_test)